# 三.批量化提取占位信息并绘制图像

# 1.批量提取每个体系的占位数据到对应文件夹下的YSITE-T_output.xlsx表格中,并给予图区的数据进行绘图

# 注:
需要更改的内容如下:

#设置根目录路径
root_directory = r'F:\MCMFData\机器学习专项\吴波\数据库'

其他:

这两部分开运行:

第一步：处理EXP文件并保存为Excel

第二步：在需要时执行绘图


1.生成占位数据文件

In [1]:
#########################################################################################################################################################################
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np


def process_exp_file(file_path):
    data = {}
    current_column = None

    with open(file_path, 'r') as file:
        for line in file:
            if '$ PLOTTED COLUMNS ARE' in line:
                match = re.search(r'Y\(FCC,([A-Za-z#0-9]+)\)', line)
                if match:
                    current_column = match.group(1)
                    if current_column not in data:
                        data[current_column] = {'T': [], 'Value': []}
            else:
                if current_column:
                    parts = line.split()
                    if len(parts) >= 2:
                        try:
                            temperature = float(parts[0])
                            value = float(parts[1])
                            data[current_column]['T'].append(temperature)
                            data[current_column]['Value'].append(np.float32(value))
                        except ValueError:
                            pass

    # 创建一个 DataFrame 列表
    dfs = []
    for col, values in data.items():
        df = pd.DataFrame(values)
        df.columns = ['T', col]
        df = df.drop_duplicates(subset='T')  # 去除重复的 'T' 值
        dfs.append(df.set_index('T'))

    # 使用 pd.concat 合并所有 DataFrame
    combined_data = pd.concat(dfs, axis=1).reset_index()

    return combined_data


def process_exp_and_save_excel(root_dir):
    # 遍历root_dir下的所有文件夹
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            # 检查文件名是否为YSITE-T.EXP（不区分大小写）
            if file.lower() == 'ysite-t.exp':
                exp_file_path = os.path.join(subdir, file)
                # 检查文件是否为空
                if os.path.getsize(exp_file_path) < 10 * 1024:
                    print(f"Skipping small file: {exp_file_path}")
                    continue  # 跳过小于10KB的文件
                print(f"Processing {exp_file_path}")
                
                # 处理EXP文件并保存为Excel
                save_exp_as_excel(exp_file_path, subdir)

def save_exp_as_excel(exp_file_path, subdir):
    processed_data = process_exp_file(exp_file_path)
    processed_data = processed_data.iloc[2:-1]
    processed_data.sort_values(by='T', ascending=True, inplace=True)
    processed_data.drop_duplicates(subset='T', inplace=True)

    # 保存处理后的数据到Excel文件
    output_excel_path = os.path.join(subdir, 'YSITE-T_output.xlsx')
    processed_data.to_excel(output_excel_path, index=False)
    print(f"Data processed and saved to {output_excel_path}")

# 设置根目录路径
root_directory = r'D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位'

# 第一步：处理EXP文件并保存为Excel
process_exp_and_save_excel(root_directory)

Processing D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\(CoCrNi)92V8\YSITE-T.EXP
Data processed and saved to D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\(CoCrNi)92V8\YSITE-T_output.xlsx
Processing D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\(CoCrNi)95V5\YSITE-T.EXP
Data processed and saved to D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\(CoCrNi)95V5\YSITE-T_output.xlsx
Processing D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\(CoCrNi)96V4\YSITE-T.EXP
Data processed and saved to D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\(CoCrNi)96V4\YSITE-T_output.xlsx
Processing D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\CoCrFeNi\YSITE-T.EXP
Data processed and saved to D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\CoCrFeNi\YSITE-T_output.xlsx
Processing D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\CoCrMnNi\YSITE-T.EXP
Data processed and saved to D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\CoCrMnNi\YSITE-T_output.xlsx
Processing D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\CoNiV\YSITE-T.EXP
Data processed and saved to D

2

形状线段画图

In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def plot_from_excel_and_save(root_dir):
    # 设置全局字体为Times New Roman
    plt.rcParams['font.family'] = 'Times New Roman'
    
    # 遍历root_dir下的所有子目录
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            # 处理所有xlsx文件
            if file.lower().endswith('.xlsx'):
                excel_file_path = os.path.join(subdir, file)
                print(f"Plotting data from {excel_file_path}")
                
                # 读取Excel文件并绘图
                plot_data_from_excel(excel_file_path, subdir)

def plot_data_from_excel(excel_file_path, subdir):
    # 读取Excel文件
    processed_data = pd.read_excel(excel_file_path)
    
    # 定义每个主元素的颜色、标记和线型
    element_styles = {
        'CO': {'color': 'blue', 'marker': 'o', 'linestyle': '-'},
        'CR': {'color': 'red', 'marker': 's', 'linestyle': '-'},
        'NI': {'color': 'green', 'marker': '^', 'linestyle': '-'},
        'V': {'color': 'orange', 'marker': 'D', 'linestyle': '-'},
        'CO#2': {'color': 'blue', 'marker': 'o', 'linestyle': '--'},
        'CR#2': {'color': 'red', 'marker': 's', 'linestyle': '--'},
        'NI#2': {'color': 'green', 'marker': '^', 'linestyle': '--'},
        'V#2': {'color': 'orange', 'marker': 'D', 'linestyle': '--'},
    }
    
    # 定义标记间隔
    mark_every = 100

    plt.figure(figsize=(12, 8))

    # 绘制每个数据系列
    for column in processed_data.columns[1:]:  # 跳过'T'列
        # 获取元素样式
        style = element_styles.get(column)
        if not style:
            # 对于未知元素，使用默认样式（黑色实线圆点）
            style = {'color': 'black', 'marker': 'o', 'linestyle': '-'}
        
        # 绘制曲线
        plt.plot(
            processed_data['T'],
            processed_data[column],
            label=column,
            color=style['color'],
            linestyle=style['linestyle'],
            marker=style['marker'],
            markevery=mark_every
        )

    # 添加图例，调整位置到右侧
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.8), fontsize=20)
    plt.xticks(fontsize=22)
    plt.yticks(fontsize=22)
    
    # 设置标题为文件名（不带扩展名）
    plt.title('FCC_CoCrNi', fontsize=25)
    
    plt.xlabel('Temperature (K)', fontsize=25)
    plt.ylabel('Site Occupancy Fractions', fontsize=25)
    
    # 调整布局以防止保存图像时切割
    plt.tight_layout()
    
    # 调整布局考虑图例
    plt.subplots_adjust(right=0.75)

    # 保存图像
    output_image_path = excel_file_path.replace('.xlsx', '.png')
    plt.savefig(output_image_path, bbox_inches='tight')
    plt.close()
    print(f"Plot saved to {output_image_path}")

# 设置根目录路径
root_directory = r'D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位'

plot_from_excel_and_save(root_directory)

2

实虚线画图

In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def plot_from_excel_and_save(root_dir):
    # 设置全局字体为Times New Roman
    plt.rcParams['font.family'] = 'Times New Roman'
    
    # 遍历root_dir下的所有子目录
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            # 处理所有xlsx文件
            if file.lower().endswith('.xlsx'):
                excel_file_path = os.path.join(subdir, file)
                print(f"Plotting data from {excel_file_path}")
                
                # 读取Excel文件并绘图
                plot_data_from_excel(excel_file_path, subdir)

def plot_data_from_excel(excel_file_path, subdir):
    # 读取Excel文件
    processed_data = pd.read_excel(excel_file_path)
    
    # 定义每个主元素的颜色、标记和线型
    #element_styles = {
    #    'CO': {'color': 'blue', 'marker': 'o', 'linestyle': '-'},'CR': {'color': 'red', 'marker': 's', 'linestyle': '-'},'NI': {'color': 'green', 'marker': '^', 'linestyle': '-'},'V': {'color': 'orange', 'marker': 'D', 'linestyle': '-'},
    #    'CO#2': {'color': 'blue', 'marker': 'o', 'linestyle': '--'},'CR#2': {'color': 'red', 'marker': 's', 'linestyle': '--'},'NI#2': {'color': 'green', 'marker': '^', 'linestyle': '--'},'V#2': {'color': 'orange', 'marker': 'D', 'linestyle': '--'},
    #}
    
    #    element_styles = {
    #    'CO': {'color': 'blue',  'linestyle': '-'},'CR': {'color': 'red',  'linestyle': '-'},'NI': {'color': 'green',  'linestyle': '-'},'V': {'color': 'orange',  'linestyle': '-'},'Fe': {'color': 'black',  'linestyle': '-'},'Mn': {'color': 'purple',  'linestyle': '-'},
    #    'CO#2': {'color': 'blue',  'linestyle': '--'},'CR#2': {'color': 'red',  'linestyle': '--'},'NI#2': {'color': 'green',  'linestyle': '--'},'V#2': {'color': 'orange',  'linestyle': '--'},'Fe#2': {'color': 'black',  'linestyle': '--'},'Mn#2': {'color': 'purple',  'linestyle': '--'}
    #}
    # 定义标记间隔
    #mark_every = 100

    # 定义每个主元素的颜色、标记和线型
    element_styles = {
        'CO': {'color': 'blue',  'linestyle': '-'},'CR': {'color': 'red',  'linestyle': '-'},'NI': {'color': 'green',  'linestyle': '-'},'V': {'color': 'orange',  'linestyle': '-'},'FE': {'color': 'black',  'linestyle': '-'},'MN': {'color': 'purple',  'linestyle': '-'},
        'CO#2': {'color': 'blue',  'linestyle': '--'},'CR#2': {'color': 'red',  'linestyle': '--'},'NI#2': {'color': 'green',  'linestyle': '--'},'V#2': {'color': 'orange',  'linestyle': '--'},'FE#2': {'color': 'black',  'linestyle': '--'},'MN#2': {'color': 'purple',  'linestyle': '--'}
    }


    plt.figure(figsize=(12, 8))

    # 绘制每个数据系列
    for column in processed_data.columns[1:]:  # 跳过'T'列
        # 获取元素样式
        style = element_styles.get(column)
        if not style:
            # 对于未知元素，使用默认样式（黑色实线圆点）
            style = {'color': 'black', 'marker': 'o', 'linestyle': '-'}
        
        # 绘制曲线
        plt.plot(
            processed_data['T'],
            processed_data[column],
            label=column,
            color=style['color'],
            linestyle=style['linestyle'],
            #marker=style['marker'],
            #markevery=mark_every
        )

    # 添加图例，调整位置到右侧
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.8), fontsize=20)
    
    # 设置x轴范围以适应数据
    min_T = processed_data['T'].min()
    max_T = processed_data['T'].max()
    #padding = (max_T - min_T) * 0.05  # 添加5%的填充
    plt.xlim(min_T , max_T )
    
    # 设置字体大小
    plt.xticks(fontsize=22, fontname='Times New Roman')
    plt.yticks(fontsize=22, fontname='Times New Roman')
    
    # 设置标题为文件名（不带扩展名）
    #plt.title('FCC-CoCrNiV0.5', fontsize=25, fontname='Times New Roman')
    
    plt.xlabel('Temperature (K)', fontsize=25, fontname='Times New Roman')
    plt.ylabel('Site Occupancy Fractions', fontsize=25, fontname='Times New Roman')
    
    # 调整布局以防止保存图像时切割
    plt.tight_layout()
    
    # 调整布局考虑图例
    plt.subplots_adjust(right=0.75)

    # 保存图像
    output_image_path = excel_file_path.replace('.xlsx', '.png')
    plt.savefig(output_image_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"Plot saved to {output_image_path}")

# 设置根目录路径
root_directory = r'D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位'

plot_from_excel_and_save(root_directory)

Plotting data from D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\(CoCrNi)92V8\YSITE-T_output.xlsx
Plot saved to D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\(CoCrNi)92V8\YSITE-T_output.png
Plotting data from D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\(CoCrNi)95V5\YSITE-T_output.xlsx
Plot saved to D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\(CoCrNi)95V5\YSITE-T_output.png
Plotting data from D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\(CoCrNi)96V4\YSITE-T_output.xlsx
Plot saved to D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\(CoCrNi)96V4\YSITE-T_output.png
Plotting data from D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\CoCrFeNi\YSITE-T_output.xlsx
Plot saved to D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\CoCrFeNi\YSITE-T_output.png
Plotting data from D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\CoCrMnNi\YSITE-T_output.xlsx
Plot saved to D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\CoCrMnNi\YSITE-T_output.png
Plotting data from D:\\Desk\\CoCrNiV论文pandat\\计算数据\\第三次重做\\占位\CoNiV\YSITE-T_output.xlsx
Plot sav